In [1]:
import Pkg; Pkg.activate("C:/Users/s151781/AppData/Local/Julia-1.3.1/GN/Project.toml")
using Revise
using Distributions
using PyPlot
using FFTW
using Compat
using WAV
using DSP
using Base64
using ForneyLab
using LinearAlgebra
using ProgressMeter

include("../functions/auxiliary/audioplayer.jl")
include("../functions/auxiliary/workflow.jl")
include("../functions/auxiliary/fourier.jl")
include("../functions/auxiliary/visualization.jl")
include("../functions/auxiliary/buffer.jl")
include("../functions/preprocessing.jl")

Activating environment at `C:\Users\s151781\AppData\Local\Julia-1.3.1\GN\Project.toml`


fft_expand (generic function with 1 method)

In [2]:
mutable struct HGF <: ForneyLab.SoftFactor
    id::Symbol
    interfaces::Vector{Interface}
    i::Dict{Symbol,Interface}

    function HGF(X, ξ; id=ForneyLab.generateId(HGF))
        
        # ensure that the input arguments are random variables
        @ensureVariables(X, ξ) 
        
        # create new object
        self = new(id, Array{Interface}(undef, 2), Dict{Symbol,Interface}())
        
        # add the node to the current factor graph
        ForneyLab.addNode!(currentGraph(), self)
        
        # add argument variables to interfaces of node
        self.i[:X] = self.interfaces[1] = ForneyLab.associate!(Interface(self), X)
        self.i[:ξ] = self.interfaces[2] = ForneyLab.associate!(Interface(self), ξ)
        
        # return object
        return self
    end
end

In [3]:
function ruleVariationalHGFOutNP(marg_X::Nothing, 
                                 marg_ξ::ProbabilityDistribution{ForneyLab.Multivariate})
    
    # caluclate required mean
    mξ = ForneyLab.unsafeMean(marg_ξ)

    # calculate required variance
    vξ = diag(ForneyLab.unsafeCov(marg_ξ))

    # calculate new parameters
    mX = zeros(size(mξ))
    vX = exp.(mξ - vξ/2)
    
    # create variational message
    return Message(ForneyLab.Multivariate, GaussianWeightedMeanPrecision, xi=mX./vX, w=diagm(1 ./vX))

end

ruleVariationalHGFOutNP (generic function with 1 method)

In [4]:
function ruleVariationalHGFIn1PN(marg_X::ProbabilityDistribution{ForneyLab.Multivariate}, 
                                 marg_ξ::Nothing)
    
    # calculate required means
    mX = ForneyLab.unsafeMean(marg_X)

    # calculate required variances
    vX = diag(ForneyLab.unsafeCov(marg_X))

    # calculate new parameters
    mξ = log.(mX.^2 + vX)
    vξ = 2.0*ones(length(mξ))

    # create variational message
    Message(ForneyLab.Multivariate, GaussianWeightedMeanPrecision, xi=mξ./vξ, w=diagm(1 ./ vξ))

end

ruleVariationalHGFIn1PN (generic function with 1 method)

In [5]:
@naiveVariationalRule(:node_type     => HGF,
                      :outbound_type => Message{GaussianWeightedMeanPrecision},
                      :inbound_types => (Nothing, ProbabilityDistribution),
                      :name          => VariationalHGFOutNP)

@naiveVariationalRule(:node_type     => HGF,
                      :outbound_type => Message{GaussianWeightedMeanPrecision},
                      :inbound_types => (ProbabilityDistribution, Nothing),
                      :name          => VariationalHGFIn1PN)

In [6]:
# number of clusters
N = 3

# length input segments
bufsize = 20

# number of frequency bins (cosine coefs only for now) TO DISCUSS
nr_freqs = 20

# create factor graph
fg = FactorGraph()

# gaussian mixture 
@RV [id=:p] p ~ ForneyLab.Dirichlet(placeholder(:p_a))
@RV [id=:z] z ~ ForneyLab.Categorical(p)
μ = Array{Variable, 1}(undef,N)
Λ = Array{Variable, 1}(undef,N)
for k = 1:3
    @RV [id=pad(:μ,k)] μ[k] ~ GaussianMeanVariance(placeholder(pad(:μ_μ,k), dims=(nr_freqs,)), placeholder(pad(:Σ_μ,k), dims=(nr_freqs,nr_freqs)))
    @RV [id=pad(:Λ,k)] Λ[k] ~ ForneyLab.Wishart(placeholder(pad(:V_Λ,k), dims=(nr_freqs,nr_freqs)), placeholder(pad(:nu_Λ,k)))
end
@RV [id=:ξ] ξ ~ GaussianMixture(z, expand([[μ[k],Λ[k]] for k = 1:N])...)

# HGF
@RV [id=:X] X ~ HGF(ξ)

# probabilistic Fourier
@RV [id=:x] x = placeholder(:C, dims=(bufsize, nr_freqs)) * X

# observation model
@RV [id=:y] y ~ GaussianMeanVariance(x, placeholder(:Σ_y, dims=(bufsize,bufsize)))
placeholder(y, :y, dims=(bufsize,))

# draw graph
ForneyLab.draw(fg)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.38.0 (20140413.2041)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 232313866234478344 -->
 232313866234478344 
 
 Dir 
 dirichlet_1 
 
<!-- 5893825738518443996 -->
 5893825738518443996 
 
 placeholder_p_a 
 
<!-- 232313866234478344--5893825738518443996 -->
 232313866234478344--5893825738518443996 
 
 p_a 
 1 out 
 2 a 
 
<!-- 15192524697899753843 -->
 15192524697899753843 
 
 W 
 wishart_2 
 
<!-- 15722247095091043343 -->
 15722247095091043343 
 
 placeholder_V_Λ_02 
 
<!-- 15192524697899753843--15722247095091043343 -->
 15192524697899753843--15722247095091043343 
 
 V_Λ_02 
 1 out 
 2 v 
 
<!-- 17985224446253792093 -->
 17985224446253792093 
 
 placeholder_nu_Λ_02 
 
<!-- 15192524697899753843--17985224446253792093 -->
 15192524697899753843--17985224446253792093 
 
 nu_Λ_02 
 1 out 
 3 nu 
 
<!-- 16245475339476901626 -->
 16245475339476901626 
 
 𝒩 
 gaussianmeanvariance_3 
 
<!-- 3241624243001868662 -->
 3241624243001868662 
 
 placeholder_Σ_μ_03 
 
<!-- 16245475339476901626--3241624243001868662 -->
 16245475339476901626--3241624243001868662 
 
 Σ_μ_03 
 1 out 
 3 v 
 
<!-- 6914700675198655069 -->
 6914700675198655069 
 
 placeholder_μ_μ_03 
 
<!-- 16245475339476901626--6914700675198655069 -->
 16245475339476901626--6914700675198655069 
 
 μ_μ_03 
 1 out 
 2 m 
 
<!-- 14514949557482810151 -->
 14514949557482810151 
 
 W 
 wishart_1 
 
<!-- 1674026535752540480 -->
 1674026535752540480 
 
 placeholder_V_Λ_01 
 
<!-- 14514949557482810151--1674026535752540480 -->
 14514949557482810151--1674026535752540480 
 
 V_Λ_01 
 1 out 
 2 v 
 
<!-- 2019970552285218052 -->
 2019970552285218052 
 
 placeholder_nu_Λ_01 
 
<!-- 14514949557482810151--2019970552285218052 -->
 14514949557482810151--2019970552285218052 
 
 nu_Λ_01 
 1 out 
 3 nu 
 
<!-- 16333339514863610719 -->
 16333339514863610719 
 
 W 
 wishart_3 
 
<!-- 15889814445358721683 -->
 15889814445358721683 
 
 placeholder_nu_Λ_03 
 
<!-- 16333339514863610719--15889814445358721683 -->
 16333339514863610719--15889814445358721683 
 
 nu_Λ_03 
 1 out 
 3 nu 
 
<!-- 2901839250376723581 -->
 2901839250376723581 
 
 placeholder_V_Λ_03 
 
<!-- 16333339514863610719--2901839250376723581 -->
 16333339514863610719--2901839250376723581 
 
 V_Λ_03 
 1 out 
 2 v 
 
<!-- 1861405483554146798 -->
 1861405483554146798 
 
 placeholder_Σ_y 
 
<!-- 7890883988266728563 -->
 7890883988266728563 
 
 placeholder_Σ_μ_02 
 
<!-- 7222437798686792102 -->
 7222437798686792102 
 
 placeholder_μ_μ_01 
 
<!-- 18161286316289710890 -->
 18161286316289710890 
 
 placeholder_y 
 
<!-- 264183357324240223 -->
 264183357324240223 
 
 𝒩 
 gaussianmeanvariance_4 
 
<!-- 18161286316289710890--264183357324240223 -->
 18161286316289710890--264183357324240223 
 
 y 
 1 out 
 1 out 
 
<!-- 2760135419844506649 -->
 2760135419844506649 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 2760135419844506649--7222437798686792102 -->
 2760135419844506649--7222437798686792102 
 
 μ_μ_01 
 1 out 
 2 m 
 
<!-- 16006742900680291198 -->
 16006742900680291198 
 
 placeholder_Σ_μ_01 
 
<!-- 2760135419844506649--16006742900680291198 -->
 2760135419844506649--16006742900680291198 
 
 Σ_μ_01 
 1 out 
 3 v 
 
<!-- 9380009781848972418 -->
 9380009781848972418 
 
 Cat 
 categorical_1 
 
<!-- 9380009781848972418--232313866234478344 -->
 9380009781848972418--232313866234478344 
 
 p 
 1 out 
 2 p 
 
<!-- 11526417578767395314 -->
 11526417578767395314 
 
 placeholder_C 
 
<!-- 3575704919801888576 -->
 3575704919801888576 
 
 × 
 multiplication_1 
 
<!-- 3575704919801888576--11526417578767395314 -->
 3575704919801888576--11526417578767395314 
 
 C